<a href="https://colab.research.google.com/github/Kkkk098/1prolongation_analysis/blob/main/prolongation_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем необходимые библиотеки.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import drive
import pprint
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

In [37]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем два набора данных.

In [38]:
prolong = pd.read_csv('/content/drive/MyDrive/Topface/prolongations.csv', encoding='utf-8')
fin = pd.read_csv('/content/drive/MyDrive/Topface/financial_data.csv', encoding='utf-8')

Первичный осмотр данных.

In [39]:
print(fin.dtypes)

id                int64
Причина дубля    object
Ноябрь 2022      object
Декабрь 2022     object
Январь 2023      object
Февраль 2023     object
Март 2023        object
Апрель 2023      object
Май 2023         object
Июнь 2023        object
Июль 2023        object
Август 2023      object
Сентябрь 2023    object
Октябрь 2023     object
Ноябрь 2023      object
Декабрь 2023     object
Январь 2024      object
Февраль 2024     object
Account          object
dtype: object


In [40]:
print(prolong.dtypes)

id        int64
month    object
AM       object
dtype: object


In [41]:
print(prolong.head())

    id        month                             AM
0   42  ноябрь 2022   Васильев Артем Александрович
1  453  ноябрь 2022   Васильев Артем Александрович
2  548  ноябрь 2022      Михайлов Андрей Сергеевич
3   87  ноябрь 2022  Соколова Анастасия Викторовна
4  429  ноябрь 2022  Соколова Анастасия Викторовна


In [42]:
print(fin.head())

    id        Причина дубля Ноябрь 2022 Декабрь 2022 Январь 2023 Февраль 2023  \
0   42                  NaN   36 220,00          NaN         NaN          NaN   
1  657  первая часть оплаты        стоп          NaN         NaN          NaN   
2  657  вторая часть оплаты        стоп          NaN         NaN          NaN   
3  594                  NaN        стоп          NaN         NaN          NaN   
4  665                  NaN   10 000,00          NaN         NaN          NaN   

  Март 2023 Апрель 2023 Май 2023 Июнь 2023 Июль 2023 Август 2023  \
0       NaN         NaN      NaN       NaN       NaN         NaN   
1       NaN         NaN      NaN       NaN       NaN         NaN   
2       NaN         NaN      NaN       NaN       NaN         NaN   
3       NaN         NaN      NaN       NaN       NaN         NaN   
4       NaN         NaN      NaN       NaN       NaN         NaN   

  Сентябрь 2023 Октябрь 2023 Ноябрь 2023 Декабрь 2023 Январь 2024  \
0           NaN          NaN       

Создаем список с названием отчетных месяцев из файла "financial_data".

In [43]:
list_of_months = fin.columns[2:-1].tolist()
print(list_of_months)

['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']


Создаем функцию для обработки значений в ячейках таблицы "financial_data". В ячейках могут встретиться

In [44]:
def clean_financial_data(df, month_cols):
    '''
    Функция приводит все значения к числу или специальным типам:
      - число - float
      - 'в ноль' - 0.0
      - 'стоп', 'stop', 'end' - 'стоп'
      - пустые - np.nan
    '''
    def parse_value(x):
        if pd.isna(x):
            return np.nan

        # Строки
        if isinstance(x, str):
            val = x.strip().lower()

            if val in ['стоп', 'stop', 'end']:
                return 'стоп'
            if val == 'в ноль':
                return 0.0

            # Число как строка
            val = val.replace('\xa0', '').replace(' ', '').replace(',', '.').replace('"', '')
            return float(val)


        # Число
        if isinstance(x, (int, float)):
            return float(x)

        return np.nan  # все остальное

    # Применяем функцию к столбцам с месяцами
    for col in month_cols:
        df[col] = df[col].apply(parse_value)

    return df

Унифицируем значения в таблице financial_data.

In [45]:
month_cols = [c for c in fin.columns if c not in ['id', 'Причина дубля', 'Account']]
fin = clean_financial_data(fin, month_cols)

print(fin.head(1))

   id Причина дубля Ноябрь 2022 Декабрь 2022  Январь 2023 Февраль 2023  \
0  42           NaN     36220.0          NaN          NaN          NaN   

  Март 2023 Апрель 2023 Май 2023 Июнь 2023 Июль 2023 Август 2023  \
0       NaN         NaN      NaN       NaN       NaN         NaN   

  Сентябрь 2023 Октябрь 2023 Ноябрь 2023  Декабрь 2023 Январь 2024  \
0           NaN          NaN         NaN           NaN         NaN   

  Февраль 2024                       Account  
0          NaN  Васильев Артем Александрович  


Создаем список необходимых нам для отчетности месяцев, в данном случае месяцев 2023 года.

In [46]:
months_2023 = [col for col in fin.columns if '2023' in col]
print(months_2023)

['Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023']


Создаем функцию для извлечения предыдущего или будущего месяца.

In [47]:
def required_month(list_of_months, month_name, idx):
    '''
    idx = [-1, 0, 1, 2]
    -1 - предыдущий месяц по списку
    0 - тот же месяц, что был на входе функции
    1 - первый месяц после отчетного
    2 - второй месяц после отчетного
    '''
    index = list_of_months.index(month_name)
    index += idx
    if index >= 0:
       month = list_of_months[index]
    else:
       month = month_name
    return month


In [48]:
def criterion(proj_df, id, month_name, list_of_months, idx=0):
    '''
    Функция для обработки ситуаций с дублями, 'стоп', NaN, 'в ноль'.
    '''

    project_rows = proj_df.loc[proj_df['id'] == id]
    month_name = required_month(list_of_months, month_name, idx)

    # Обработка дублей проекта
    if len(project_rows) > 1:

        # Собираем части отгрузки для проекта в список
        values = project_rows[month_name].tolist()

        '''
        Если встречаем 'стоп', то не учитываем проект ни в числителе,
        ни в знаменатиле коэффициента пролонгации
        '''
        if any(v == 'стоп' for v in values):
            return 0.0

        # Если все части оплаты равны 0, то смотрим на предыдущий месяц
        elif all(v == 0 for v in values):
            month_name = required_month(list_of_months, month_name, -1)
            values = project_rows[month_name].tolist()

            '''
            В предыдущем месяце будут либо нули, либо реальные отгрузки,
            либо в одной из частей оплаты просто пустая ячейка, поэтому достаточно
            вернуть сумму значений, игнорируя NaN
            '''
            return np.nansum([v for v in values])


        else:
            '''
            Если в наших частях оплаты не встречается 'стоп' или оба нуля, то достаточно
            вернуть сумму значений. Даже если будет оба NaN, что говорит о преждевременном
            завершении проекта и где-то в предыдущих месяцах должен встретиться 'стоп',
            наша возвращаемая сумма будет равна 0 и этот проект не повлияет на коэфф-ты
            ни в числителе, ни в знаменатиле
            '''
            return np.nansum([v for v in values])

    # Обработка проекта без дублей
    elif len(project_rows) == 1:
        value = project_rows[month_name].iloc[0]
        '''
        Если встречаем 'стоп', то не учитываем проект ни в числителе,
        ни в знаменатиле коэффициента пролонгации
        '''
        if value == 'стоп':
            return 0.0

        elif np.isnan(value):
            '''
            NaN говорит о закрытии проекта раньше срока, в предыдущем месяце должен
            встретиться 'стоп', поэтому не учитываем проект
            '''
            return 0.0

        # Смотрим на предыдущий месяц
        elif value == 0:
            month_name = required_month(list_of_months, month_name, -1)
            value = project_rows[month_name].iloc[0]
            return np.nansum(value)

        # Из перебора возможных значений ячейки остается только число - сумма отгрузки
        elif value != 0:
            return value


In [49]:
def calculation_k(manager_name, month_name, list_of_months, k):
    '''
    Функция для подсчета коэффициентов пролонгации.
    k - месяц (первый, второй), для которого необходимо посчитать коэффициент.
    denominator - знаменатель для расчета коэффициента сотрудника
    numerator - числитель
    denominator_general - знаменатель для подсчета коэффициента по отделу
    numerator_general - числитель для подсчета коэффициента по отделу
    '''

    denominator = 0
    numerator = 0
    denominator_general = 0
    numerator_general = 0

    filtered_df = prolong[(prolong['AM'] == manager_name) & (prolong['month'] == month_name.lower())]

    '''
    Если в конкретном месяце у сотрудника по таблице "prolongations" не было
    завершений, то для сотрудника возвращаем прочерк (NaN), а в коэффициенте
    для отдела не учитываем.
    '''
    if filtered_df.empty:
        return (np.nan, 0.0, 0.0)

    # Список проектов (id), завершенных в определенный месяц у конкретного менеджера
    list_of_proj = filtered_df['id'].unique().tolist()

    '''
    Из таблицы "financial_data" выбираем строчки для рассматриваемых закрытых
    проектов у конкретного менеджера.
    '''
    proj_df = fin[fin['id'].isin(list_of_proj)]

    # Проходимся по каждому проекту
    for id in list_of_proj:

        denominator += criterion(proj_df, id, month_name, list_of_months)
        denominator_general += denominator

        numerator += criterion(proj_df, id, month_name, list_of_months, k)
        numerator_general += numerator

    # Учитываем деление на ноль
    if denominator == 0:

        return (0.0, 0.0, 0.0)
    else:
        # Коэффициент находим как отношение одной суммы к другой
        return (numerator / denominator, denominator_general, numerator_general)


In [50]:
# Расчёт коэффициентов

# Словарь для коэффициентов для каждого сотрудника
prolong_dict = {}

# Словарь для значений по отделу
dep_dict = {m: {'den_gen_1': 0,
                'num_gen_1': 0,
                'den_gen_2': 0,
                'num_gen_2': 0} for m in months_2023}

# Находим массив ФИО аккаунт-менеджеров
managers = prolong['AM'].unique()

# Проходимся по каждому сотруднику из таблицы "prolongations"
for manager_name in managers:

    prolong_dict[manager_name] = {}

    for month_name in months_2023:

        k_1, den_gen_1, num_gen_1 = calculation_k(manager_name, month_name, list_of_months, k=1)
        k_2, den_gen_2, num_gen_2 = calculation_k(manager_name, month_name, list_of_months, k=2)

        prolong_dict[manager_name][month_name] = {'K1': k_1, 'K2': k_2}
        dep_dict[month_name]['den_gen_1'] += den_gen_1
        dep_dict[month_name]['num_gen_1'] += num_gen_1
        dep_dict[month_name]['den_gen_2'] += den_gen_2
        dep_dict[month_name]['num_gen_2'] += num_gen_2


In [51]:
department_dict = {}

# Рассчитываем коэффициенты пролонгации для всего отдела
for m in months_2023:
    den1 = dep_dict[m]['den_gen_1']
    num1 = dep_dict[m]['num_gen_1']
    den2 = dep_dict[m]['den_gen_2']
    num2 = dep_dict[m]['num_gen_2']

    department_dict[m] = {
        'K1 для отдела': num1 / den1 if den1 != 0 else 0,
        'K2 для отдела': num2 / den2 if den2 != 0 else 0
    }

In [52]:
pprint.pprint(prolong_dict)

{'Васильев Артем Александрович': {'Август 2023': {'K1': np.float64(0.172432592799243),
                                                  'K2': np.float64(0.23000851226824104)},
                                  'Апрель 2023': {'K1': np.float64(0.4492227929866154),
                                                  'K2': np.float64(0.5346704526366209)},
                                  'Декабрь 2023': {'K1': np.float64(0.5354083873128556),
                                                   'K2': np.float64(0.6585690226511197)},
                                  'Июль 2023': {'K1': np.float64(0.47750385208012325),
                                                'K2': np.float64(0.39078582434514636)},
                                  'Июнь 2023': {'K1': np.float64(0.5850156702127803),
                                                'K2': np.float64(0.5826319677723358)},
                                  'Май 2023': {'K1': np.float64(0.38928703566102807),
                                 

In [53]:
'''
Посчитаем для каждого менеджера количество месяцев по таблице "prolongations",
в которых были закрытия проектов. Это будет знаменатель, необходимый для
подсчета средних за год коэффициентов по каждому сотруднику.
'''

num_dict = {}

for manager_name in managers:
    num = prolong.loc[prolong['AM'] == manager_name, 'month'].unique()
    num = [m for m in num if '2023' in m]
    num_dict[manager_name] = len(num)

print(num_dict)

{'Васильев Артем Александрович': 12, 'Михайлов Андрей Сергеевич': 10, 'Соколова Анастасия Викторовна': 12, 'Иванова Мария Сергеевна': 8, 'Попова Екатерина Николаевна': 11, 'Смирнова Ольга Владимировна': 10, 'Кузнецов Михаил Иванович': 7, 'без А/М': 2, 'Петрова Анна Дмитриевна': 1, 'Федорова Марина Васильевна': 1}


In [54]:
'''
Среднее по каждому сотруднику за год находим следующим образом:
суммируем коэффициенты за все месяцы и делим на число месяцев, в которых
у сотрудника были закрытия по плану
'''
for manager_name in managers:
    k1_sum = np.nansum([v['K1'] for v in prolong_dict[manager_name].values()])
    k2_sum = np.nansum([v['K2'] for v in prolong_dict[manager_name].values()])
    k_1_y = k1_sum / num_dict[manager_name]
    k_2_y = k2_sum / num_dict[manager_name]

    # Добавляем годовой показатель для каждого менеджера
    prolong_dict[manager_name]['year 2023'] = {'K1': k_1_y, 'K2': k_2_y}


Визуально проверяем полученные данные.

In [55]:
pprint.pprint(prolong_dict)

{'Васильев Артем Александрович': {'year 2023': {'K1': np.float64(0.5575069466701552),
                                                'K2': np.float64(0.5750715652657593)},
                                  'Август 2023': {'K1': np.float64(0.172432592799243),
                                                  'K2': np.float64(0.23000851226824104)},
                                  'Апрель 2023': {'K1': np.float64(0.4492227929866154),
                                                  'K2': np.float64(0.5346704526366209)},
                                  'Декабрь 2023': {'K1': np.float64(0.5354083873128556),
                                                   'K2': np.float64(0.6585690226511197)},
                                  'Июль 2023': {'K1': np.float64(0.47750385208012325),
                                                'K2': np.float64(0.39078582434514636)},
                                  'Июнь 2023': {'K1': np.float64(0.5850156702127803),
                                 

In [56]:
'''
Подсчитываем годовой показатель для работы всего отдела. Для этого суммируем
коэффициенты за все месяцы и делим на число месяцев в году.
'''
department_dict['year 2023 по отделу'] = {
    'K1 для отдела': np.nanmean([v['K1 для отдела'] for v in department_dict.values()]),
    'K2 для отдела': np.nanmean([v['K2 для отдела'] for v in department_dict.values()])
}

In [57]:
pprint.pprint(department_dict)

{'year 2023 по отделу': {'K1 для отдела': np.float64(0.5351085428818423),
                         'K2 для отдела': np.float64(0.5532187958498135)},
 'Август 2023': {'K1 для отдела': np.float64(0.37557513126106123),
                 'K2 для отдела': np.float64(0.36677745568158093)},
 'Апрель 2023': {'K1 для отдела': np.float64(0.5024569419778503),
                 'K2 для отдела': np.float64(0.5588599689742404)},
 'Декабрь 2023': {'K1 для отдела': np.float64(0.44909566360785524),
                  'K2 для отдела': np.float64(0.5280919474441969)},
 'Июль 2023': {'K1 для отдела': np.float64(0.4755098473993756),
               'K2 для отдела': np.float64(0.4739941173179701)},
 'Июнь 2023': {'K1 для отдела': np.float64(0.4696605991071381),
               'K2 для отдела': np.float64(0.45344289478720545)},
 'Май 2023': {'K1 для отдела': np.float64(0.34584859363195636),
              'K2 для отдела': np.float64(0.40330943134377517)},
 'Март 2023': {'K1 для отдела': np.float64(0.44220798967639

Последняя часть - импорт посчитанных данных в таблицу эксель-формата.

In [58]:
# Преобразуем данные по сотрудникам
rows = []
for manager, months in prolong_dict.items():
    for month, values in months.items():
        rows.append({
            'Сотрудник': manager,
            'Месяц': month,
            'K1': values.get('K1', np.nan),
            'K2': values.get('K2', np.nan)
        })

df_managers = pd.DataFrame(rows)

# Преобразуем данные по отделу
rows_dep = []
for month, values in department_dict.items():
    rows_dep.append({
        'Сотрудник': 'Отдел',
        'Месяц': month.replace(' по отделу', ''),
        'K1': values.get('K1 для отдела', np.nan),
        'K2': values.get('K2 для отдела', np.nan)
    })

df_department = pd.DataFrame(rows_dep)

# Объединяем показатели работы для менеджеров и по отделу
df_full = pd.concat([df_managers, df_department], ignore_index=True)

# Создадим отдельные таблицы по каждому коэффициенту
months_order = [
    'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023',
    'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
    'Ноябрь 2023', 'Декабрь 2023', 'year 2023'
]

# Таблица для K1
df_k1 = df_full.pivot(index='Сотрудник', columns='Месяц', values='K1')
df_k1 = df_k1.reindex(columns=[m for m in months_order if m in df_k1.columns])
df_k1 = df_k1.round(2)

# Таблица для K2
df_k2 = df_full.pivot(index='Сотрудник', columns='Месяц', values='K2')
df_k2 = df_k2.reindex(columns=[m for m in months_order if m in df_k2.columns])
df_k2 = df_k2.round(2)

# Сохраняем обе таблицы на одном листе
output_file = 'коэффициенты_K1_K2.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Создаем пустой лист
    pd.DataFrame().to_excel(writer, index=False, sheet_name='Коэффициенты')
    ws = writer.sheets['Коэффициенты']

    ws.append(["Таблица K1"])
    for r in dataframe_to_rows(df_k1.reset_index(), index=False, header=True):
        ws.append(r)

    ws.append([])

    ws.append(["Таблица K2"])
    for r in dataframe_to_rows(df_k2.reset_index(), index=False, header=True):
        ws.append(r)

    writer._save()

wb = load_workbook(output_file)
ws = wb.active

for cell in ws["1:1"]:
    cell.font = Font(bold=True)

for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center')

for cell in ws[1]:
    cell.font = Font(bold=True)
for row in range(1, ws.max_row+1):
    if ws.cell(row, 1).value in ("Таблица K1", "Таблица K2"):
        ws.cell(row, 1).font = Font(bold=True)
        ws.merge_cells(start_row=row, start_column=1, end_row=row, end_column=len(df_k1.columns)+1)

wb.save(output_file)

print("Файл 'коэффициенты_K1_K2.xlsx' готов.")

Файл 'коэффициенты_K1_K2.xlsx' готов.
